# <center> Get user graph for `Russian`

In [1]:
import sys
import util
import json
import time
from twython import Twython

### Twitter API credentials

In [7]:
APP_KEY = "AV4KujYNS1y6HdmQ4zXxHw2Tx"
APP_SECRET = "xd2FuFaS5zE1ON407C3U5X0kdEtRJIjH9OMDmgyMmxQCZMFlli"
OAUTH_TOKEN = "1058011325316636672-HkpzhyiOHdJkuwi3FD9QrkMFbuHuVt"
OAUTH_TOKEN_SECRET = "iiD75H9ERtkATnd74pVRBfM7TI189BSmDasYzN2uUO123"

### Retrieve user graph

Get user set.

In [8]:
users = set(util.get_authors("../Datasets/russian_rtid.txt").keys())

Get the followgraph of users present in the trace.

In [9]:
# init
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
FollowGraph = dict()
outfile = "../Datasets/russian_adjList.txt"

for u in users:
    
    # reinit cursor
    cursor = -1
    
    # while there are followers left for this user
    while cursor != 0:
        retry = True
        
        # if we haven't this page of followers yet
        while retry:
            try:
                result = twitter.get_followers_ids(id=u, cursor=cursor)
                with open(outfile, 'a') as out:
                    for v in result['ids']:
                        if v in users:
                            out.write("{} {}\n".format(u,v))
                cursor = result['next_cursor']
                retry = False
                
            # twitter api rate limit exceeded --> sleep then reconnect and retry
            except:
                reset_time = float(twitter.get_lastfunction_header('x-rate-limit-reset'))
                del twitter
                while time.time() <= reset_time:
                    time_left = reset_time - time.time()
                    sys.stdout.flush()
                    sys.stdout.write("Sleeping... {:.0f}min {:.0f}sec left\r".format(time_left//60, time_left%60))
                    time.sleep(1)
                twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
                retry = True

TwythonError: This function must be called after an API call.                                It delivers header information.

UNUSED Function to get set of followers from user id and write to outfile.

In [4]:
def get_followers(uid, outfile):
    
    # init
    twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    followers_set = set()
    cursor = -1
    
    # while there are followers left for this user
    while cursor != 0:
        retry = True
        
        # if we haven't this page of followers yet
        while retry:
            try:
                result = twitter.get_followers_ids(id=uid, cursor=cursor)
                followers_set = followers_set.union(result['ids'])
                cursor = result['next_cursor']
                retry = False
                
            # twitter api rate limit exceeded --> sleep then reconnect and retry
            except:
                reset_time = float(twitter.get_lastfunction_header('x-rate-limit-reset'))
                del twitter
                while time.time() <= reset_time:
                    time_left = reset_time - time.time()
                    sys.stdout.flush()
                    sys.stdout.write("Sleeping... {:.0f}min {:.0f}sec left\r".format(time_left//60, time_left%60))
                    time.sleep(1)
                twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
                retry = True
                
    # return set of followers
    return followers_set

### Write adj list to out

In [ ]:
out = open("../Datasets/russian_adjList.txt", "w")

for u in FollowGraph:
    for v in FollowGraph[u]:
        out.write("{} {}\n".format(u,v))
        
out.close()       

## Old

In [ ]:
FollowGraph = dict()

for line in open("../Datasets/russian_rtid.txt"):
    line = line.split()
    uid, rtid = int(line[2]), int(line[3])
    
    # get followgraph for uid
    if uid not in FollowGraph:
        FollowGraph[uid] = get_followers(uid).intersection(users)
    
    # get followgraph for rtid
    if rtid != -1 and rtid in Author:
        rtu = Author[rtid]
        if rtu not in FollowGraph:
            FollowGraph[rtu] = get_followers(uid).intersection(users)